In [2]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime

In [3]:
def aggregate_data(
    plays_fname: str, player_play_fname: str, players_fname: str, tracking_fname_list: list
) -> pd.DataFrame:
    """
    Create the aggregate dataframe by merging together the plays data, tracking data, and tackles data
    In the aggregate dataframe, the tackles will be represented by a tackle_dict column. Each entry is a
    dict which consists of the nflId of the tackler as the key, and a tuple of (tackles, assists) as the value

    :param plays_fname: the filename of the plays data
    :param tackle_fname: the filename of the tackles data
    :param players_fname: the filename of the players data
    :param tracking_fname_list: a list of filenames of all tracking data

    :return df_agg3: the aggregate dataframe
    """
    print(
        "INFO: Aggregating data from play data, tracking data, tackles data, and players data into a master dataframe..."
    )
    # import files
    df_plays = pd.read_csv(plays_fname)
    df_tracking = pd.concat(
        [pd.read_csv(tracking_fname) for tracking_fname in tracking_fname_list]
    )
    df_players = pd.read_csv(players_fname)
    df_player_play = pd.read_csv(player_play_fname)
    # Create column for age
    # Birthdays have NAs but we'll ignore those for now.
    # df_players["birthDate"] = pd.to_datetime(
    #     df_players["birthDate"], format="%Y-%m-%d", errors="coerce"
    # )
    # df_players["age"] = (
    #     datetime.now().year
    #     - df_players["birthDate"].dt.year
    #     + (datetime.now().month - df_players["birthDate"].dt.month) / 12
    # )
    # df_players["age"] = df_players["age"].astype("float32")
    # df_players["age"] = df_players["age"].fillna(df_players["age"].mean())
    # aggregate plays, tracking, players tables
    df_agg1 = pd.merge(df_plays, df_tracking, on=["gameId", "playId"], how="inner")
    df_agg2 = pd.merge(
        df_agg1, df_player_play, on=["gameId", "playId", "nflId"], how="inner"
    )  # how = inner will drop any nflId and displayNames that are not common in both dataframes. For example the "football" rows are dropped.
    # df_agg3 = pd.merge(
    #     df_agg2, df_players, on = ["nflId"], how = "inner"
    # )
    return df_agg2

In [4]:
tracking_list = ['data/tracking_week_1.csv', 'data/tracking_week_2.csv', 'data/tracking_week_3.csv', 'data/tracking_week_4.csv', 'data/tracking_week_5.csv', 'data/tracking_week_6.csv', 'data/tracking_week_7.csv',
'data/tracking_week_8.csv', 'data/tracking_week_9.csv']
combined_df = aggregate_data('data/plays.csv', 'data/player_play.csv', 'data/players.csv', ['data/tracking_week_1.csv'])

INFO: Aggregating data from play data, tracking data, tackles data, and players data into a master dataframe...


In [5]:
def rotate_direction_and_orientation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rotate the direction and orientation angles so that 0° points from left to right on the field, and increasing angle goes counterclockwise
    This should be done BEFORE the call to make_plays_left_to_right, because that function with compensate for the flipped angles.

    :param df: the aggregate dataframe created using the aggregate_data() method

    :return df: the aggregate dataframe with orientation and direction angles rotated 90° clockwise
    """
    print(
        "INFO: Transforming orientation and direction angles so that 0° points from left to right, and increasing angle goes counterclockwise..."
    )
    df["o_clean"] = (-(df["o"] - 90)) % 360
    df["dir_clean"] = (-(df["dir"] - 90)) % 360
    return df


def make_plays_left_to_right(df: pd.DataFrame) -> pd.DataFrame:
    """
    Flip tracking data so that all plays run from left to right. The new x, y, s, a, dis, o, and dir data
    will be stored in new columns with the suffix "_clean" even if the variables do not change from their original value.

    :param df: the aggregate dataframe created using the aggregate_data() method

    :return df: the aggregate dataframe with the new columns such that all plays run left to right
    """
    print("INFO: Flipping plays so that they all run from left to right...")
    df["x_clean"] = np.where(
        df["playDirection"] == "left",
        120 - df["x"],
        df[
            "x"
        ],  # 120 because the endzones (10 yds each) are included in the ["x"] values
    )
    df["y_clean"] = df["y"]
    df["s_clean"] = df["s"]
    df["a_clean"] = df["a"]
    df["dis_clean"] = df["dis"]
    df["o_clean"] = np.where(
        df["playDirection"] == "left", 180 - df["o_clean"], df["o_clean"]
    )
    df["o_clean"] = (df["o_clean"] + 360) % 360  # remove negative angles
    df["dir_clean"] = np.where(
        df["playDirection"] == "left", 180 - df["dir_clean"], df["dir_clean"]
    )
    df["dir_clean"] = (df["dir_clean"] + 360) % 360  # remove negative angles
    return df

def label_run_or_pass(df: pd.DataFrame) -> pd.DataFrame:
    print("INFO: Labeling plays as runs or passes")
    df["is_run"] = pd.isna(df["passResult"]) | (df["passResult"] == "R")
    return df

In [6]:
def select_frames(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove frames during and after the ball is snapped.

    Args:
        df (pd.DataFrame): DataFrame of tracking plays

    Returns:
        pd.DataFrame: DataFrame with frames during and after snap are removed
    """
    print("INFO: Removing inactive frames...")
    before = len(df)
    df = df[df['frameType'] == 'BEFORE_SNAP']
    after = len(df)
    print(f"INFO: {before - after} rows removed")
    return df

def select_dropbacks(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove non-dropback plays.

    Args:
        df (pd.DataFrame): DataFrame of tracking plays

    Returns:
        pd.DataFrame: DataFrame with isDropback = True
    """
    print("INFO: Removing irrelevant plays...")
    before = len(df)
    df = df[df['isDropback'] == True]
    after = len(df)
    print(f"INFO: {before - after} rows removed")
    return df

In [7]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes as input the aggregated dataframe of plays, tackles, players, and tracking data and performs
    the following preprocessing operations:

    1) Rotates the direction and orientation data so that the convention matches the unit circle
    2) Flips plays so that they run from left to right
    3) Adds a label to indicate whether the play is a pass or a run

    Subsequently, it cleans the data as follows:

    1) Remove plays with penalties
    2) Remove plays that resulted in touchdowns
    3) Convert teams from strings to ints to reduce memory
    4) Remove inactive frames (before the ball snap and after the tackle)
    5) Remove any bad data (not all players are tracked, multiple ballcarriers)
    6) Strip unused df columns to save memory

    :param df (pd.DataFrame): the original, aggregated dataframe
    :return df_clean (pd.DataFrame): the cleaned dataframe
    """

    # Data preprocessing so that all plays run from left-to-right and all angles match the standard unit circle convention
    df = rotate_direction_and_orientation(df)
    df = make_plays_left_to_right(df)
    df = label_run_or_pass(df)

    # Data cleaning
    df = select_frames(df)
    df = select_dropbacks(df)

    return df

In [8]:
combined_df = clean_data(combined_df)

INFO: Transforming orientation and direction angles so that 0° points from left to right, and increasing angle goes counterclockwise...
INFO: Flipping plays so that they all run from left to right...
INFO: Labeling plays as runs or passes
INFO: Removing inactive frames...
INFO: 2350304 rows removed
INFO: Removing irrelevant plays...
INFO: 1671802 rows removed


In [9]:
len(combined_df)

2773694

In [22]:
combined_df.columns[100:]

Index(['inMotionAtBallSnap', 'shiftSinceLineset', 'motionSinceLineset',
       'wasRunningRoute', 'routeRan', 'blockedPlayerNFLId1',
       'blockedPlayerNFLId2', 'blockedPlayerNFLId3',
       'pressureAllowedAsBlocker', 'timeToPressureAllowedAsBlocker',
       'pff_defensiveCoverageAssignment',
       'pff_primaryDefensiveCoverageMatchupNflId',
       'pff_secondaryDefensiveCoverageMatchupNflId', 'o_clean', 'dir_clean',
       'x_clean', 'y_clean', 's_clean', 'a_clean', 'dis_clean', 'is_run'],
      dtype='object')

In [17]:
combined_df['routeRan'].unique()

array([nan, 'OUT', 'SLANT', 'SCREEN', 'CROSS', 'GO', 'FLAT', 'HITCH',
       'IN', 'CORNER', 'POST', 'ANGLE', 'WHEEL'], dtype=object)

In [18]:
combined_df['offenseFormation'].unique()

array(['SHOTGUN', 'EMPTY', 'SINGLEBACK', 'I_FORM', 'PISTOL', nan, 'JUMBO'],
      dtype=object)

In [19]:
combined_df['receiverAlignment'].unique()

array(['3x1', '4x1', '2x2', '2x1', '3x2', nan, '1x0', '1x1', '2x0'],
      dtype=object)

In [20]:
combined_df['pff_passCoverage'].unique()

array(['Cover-0', 'Cover-2', 'Cover-3', 'Cover-6 Right', 'Red Zone',
       'Quarters', 'Cover-1', 'Cover-3 Seam', 'Cover 6-Left', nan,
       '2-Man', 'Cover-3 Cloud Right', 'Bracket', 'Prevent', 'Goal Line',
       'Cover-3 Double Cloud', 'Cover-3 Cloud Left'], dtype=object)

In [21]:
combined_df['pff_manZone'].unique()

array(['Man', 'Zone', 'Other', nan], dtype=object)

In [23]:
combined_df['inMotionAtBallSnap'].unique()

array([False, nan, True], dtype=object)

In [24]:
combined_df['shiftSinceLineset'].unique()

array([False, nan, True], dtype=object)

In [26]:
combined_df['motionSinceLineset'].unique()

array([False, nan, True], dtype=object)